In [123]:
import csv, os, re, shutil,json, sys
import pandas as pd 
TCP = '/Users/amycweng/DH/TCP'
sys.path.append('../')

def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

### Find texts that contain sermons (DIV tag with attribute "TYPE"='sermon' or containing key terms in the title or subject headings)

In [182]:
from bs4 import BeautifulSoup, SoupStrainer
def is_sermon(filepath):
    # read the input XML file 
    with open(filepath,'r') as file: 
        data = file.read()
    # use soupstrainer to only parse the main body
    tag = SoupStrainer("DIV1")
    soup = BeautifulSoup(data,features="xml",parse_only=tag)
    sermons = soup.findAll(attrs={"TYPE": re.compile("^(?!list|reply|catalogue|calendar)(sermon)")})
    if len(sermons) > 0:      
        return True
    return False

def get_lang(filepath): 
    # read the input XML file 
    with open(filepath,'r') as file: 
        data = file.read()
    # use soupstrainer to only parse the main body
    tag = SoupStrainer("LANGUSAGE")
    soup = BeautifulSoup(data,features="xml",parse_only=tag)
    return soup.text

In [ ]:
metadataFolder = '/Users/amycweng/DH/ECBC-Data-2022/TCP metadata'
sermons = []
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    for idx,tcpID in enumerate(data['id']):
        if tcpID == "id": continue
        filepath = findTextTCP(tcpID)
        subject_headings = data['keywords'][idx].replace(" -- ","; ").replace("  "," ")
        if is_sermon(filepath): 
            sermons.append( {"id": tcpID, 
                            "estc":data['estc'][idx],
                            "stc":data['stc'][idx],
                            "title": data['title'][idx],
                            "authors": "; ".join(set(data['author'][idx].split("; "))),
                            "publisher": data['publisher'][idx],
                            "pubplace":data['pubplace'][idx],
                            "subject_headings":subject_headings,
                            "date":data['date'][idx]
                            }
            )
    print(csvFile)

In [ ]:
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    for idx,tcpID in enumerate(data['id']):
        if tcpID == "id": continue
        filepath = findTextTCP(tcpID)
        subject_headings = data['keywords'][idx].replace(" -- ","; ").replace("  "," ")
        if re.search('sermon', subject_headings.lower()): 
            if is_sermon(filepath):
                sermons.append( {"id": tcpID, 
                                "estc":data['estc'][idx],
                                "stc":data['stc'][idx],
                                "title": data['title'][idx],
                                "authors": "; ".join(set(data['author'][idx].split("; "))),
                                "publisher": data['publisher'][idx],
                                "pubplace":data['pubplace'][idx],
                                "subject_headings":subject_headings,
                                "date":data['date'][idx]
                                }
                )
    print(csvFile)

In [152]:
sermons = pd.DataFrame(sermons)
sermons.to_csv("../assets/sermons.csv",index=False)
print(f"{len(sermons)} TCP XML files contain sermons.") 
# 399 without sermons in their subject headings 
# 4253 total TCP XML files contain sermons 

4253 TCP XML files contain sermons.


In [169]:
sermons_metadata = pd.read_csv("../assets/sermons.csv")
sermons = sermons_metadata.to_dict(orient='records')
sermons = {entry['id']: entry for entry in sermons}

## Exclude foreign language texts

In [166]:
# Find the texts that most likely contain sermons but do not have the proper div tag in the XML 
texts = {}

by_subj, by_title, by_both = [],[],[]
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    for idx,tcpID in enumerate(data['id']):
        if tcpID not in sermons: 
            title = data['title'][idx]
            clean_title = title.lower().replace("'",'')
            subject_headings = data['keywords'][idx]
            unique_subjects = " -- ".join(set(subject_headings.split(" -- ")))
            subject_hit, title_hit = False, False
            
            if re.search('sermon', subject_headings.lower()): 
                subject_hit = True 
            if re.search(r'sermon|preached|preacht|preachd', clean_title): 
                title_hit = True
            if subject_hit and title_hit: 
                by_both.append(tcpID)
            elif subject_hit: 
                by_subj.append(tcpID)
            elif title_hit: 
                by_title.append(tcpID)
            if subject_hit or title_hit:  
                texts[tcpID] =  {"id": tcpID, 
                                "estc":data['estc'][idx],
                                "stc":data['stc'][idx],
                                "title": data['title'][idx],
                                "authors": "; ".join(set(data['author'][idx].split("; "))),
                                "publisher": data['publisher'][idx],
                                "pubplace":data['pubplace'][idx],
                                "subject_headings":unique_subjects,
                                "date":data['date'][idx]
                                }
print(len(texts))

1776


In [167]:
len(by_both), len(by_subj), len(by_title)

(1075, 312, 389)

In [ ]:
foreign = {}
foreign_missing = {}
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    for idx,tcpID in enumerate(data['id']):
        if tcpID in sermons: 
            continue
            filepath = findTextTCP(tcpID)
            tcpIDlang = get_lang(filepath)
            if tcpIDlang != 'eng' and tcpIDlang != "lat": 
                foreign[tcpID] = tcpIDlang
        elif tcpID in texts: 
            filepath = findTextTCP(tcpID)
            tcpIDlang = get_lang(filepath)
            if tcpIDlang != 'eng' and tcpIDlang != "lat": 
                foreign_missing[tcpID] = tcpIDlang
    print(csvFile)

## Items to exclude

In [402]:
exclude = ['B01388', 'A89416', 'B28836', 'B26622', 'B12473', 'B03839', 
           'B06138', 'B09463', 'A89104', 'A83012', 'A81906', 'A84063', 
           'A81606', 'A88596', 'A81417', 'A85341', 'A90702', 'A96864', 
           'A93332', 'A90476', 'A95939', 'A79568', 'A73832', 'A77100', 
           'A63877', 'A62992', 'A67411', 'A60864', 'A64639', 'A64197', 
           'A61683', 'A48191', 'A43685', 'A43806', 'A46883', 'A40538', 
           'A41496', 'A45149', 'A48968', 'A56791', 'A55289', 'A54939', 
           'A57258', 'A56278', 'A58892', 'A16497', 'A18267', 'A03696', 
           'A06013', 'A31039', 'A60568', 'A64394', 'A68546', 'A65419', 
           'A77638', 'A78013', 'A42577', 'A47973', 'A42539', 'A45574', 
           'A00164', 'A00156', 'A09418', 'A11848', 'A18019', 'A13299', 
           'A14927', 'A30903', 'A36190', 'A31459', 'A26859', 'A26426', 
           'A26065']

# foreign texts from the XMLs with sermons 
exclude_foreign = {'A95346': 'wel', 'B34542': 'fre', 'B10302': 'wel', 
                   'B02801': 'fre', 'B07519': 'wel', 'A87349': 'wel', 
                   'A81783': 'fre', 'A81315': 'wel', 'A91720': 'sco', 
                   'A95629': 'fre', 'A91479': 'fre', 'A91847': 'fre', 
                   'A62564': 'fre', 'A52002': 'fre', 'A29335': 'fre', 
                   'A29334': 'fre', 'A31429': 'new', 'A64645': 'wel', 
                   'A76482': 'wel', 'A72359': 'wel', 'A00687': 'sco', 
                   'A17050': 'sco'}
# foreign texts from the XMLs without sermon sections 
exclude_foreign_missing = {'A95720': 'wel', 'B04329': 'wel', 'B09870': 'fre', 
                           'A60308': 'fre', 'A50422': 'wel', 'A59547': 'fre', 
                           'A29332': 'fre'}

exclude_annotated = [
    'A96361', # commentary on the Sermon on the Mount 
    'A16204', # only a title page & colophon of a 16th cent. sermon
    'A53661', # remarks on a sermon 
    'A42786', # remarks on remarks on a sermon
    'A43674', # discourses upon a funeral sermon
    'A17423', # musical compositions (even tho title mentions a sermon)
    'B03994', # letter w/ an account of some preacher w/ hymns that they sing
    'A76964', # catechism; does not contain the sermon promised in the title
    'A68730', # does not actually contain the sermon promised in the title 
    'B03688', # narrative account
    'A31468', # censure
    'A15864', # guide to hearing sermons first written in Latin
    'A34897', # reply to sermon
    'B17774', # Psalms 
    'A13551', # chapter 
    'A52900', # response to sermon
    'A52284', # letters & hymns sung at a funeral sermon
    'A50287', # petition, embassage
    'A96439', # Quaker letters & remarks 
    'A39120', # reply to sermon
    'A53461', # broadside mock sermon
    'A22477', # royal proclamation
    'A33382', # book catalogue 
    'A62876', # dissenting polemic
    'A86098', # speech against a sermon
    'A83515', # Tho. Edwards' Gangraena 
    'A53708', # meditations & discourses 
    'A34396', # mock sermon
    'A40761', # Quaker testimony 
    'A03927', # reasons & answers about the book of common prayer 
    'A10581', # dialogue
    'A49230', # reply to a treatise 
    'A35266', # catalogue of writers 
    'A32938', # articles of enquiry in Diocese of Carlisle; visitation sermon
    'A86442', # Quaker reply to sermon
    'A01012', # response to an accident that occurred at a Catholic sermon 
    'A35017', # criticism of Scotch sermons
    'A83979', # monsters & accounts of portentous events; brief relation of her funeral sermon
    'A82859', # parliamentary_declaration
    'A14186', # Psalms
    'A07105', # treatise about the schism
    'A26870', # actual funeral sermon is missing from the microfilm 
    'A09426', # William Perkins' The foundation of Christian religion; 'religious_tract', not labeled as a sermon & no place of preaching 
    'A20733', # Downame's defense of his sermon
    'A10341', # reply to Downame's defense 
    'A68172', # critique of a 'wicked sermon',
    'A44239', # discourse referring back to an earlier printed sermon
    'A52641', # primarily narrative of death & burial; funeral sermon hard to extract and identify 
    'A74862', # letter exposing how a writer contradicted with his past sermons 
    'A97284', # primarily a narrative & history about the Dutch fleet for the Commonwealth 
    'A84987', # mineral waters in Germany 
    'A61076', # response to 'a late scurrilous libel, prefix'd to a sermon preach'd nine and thirty years ago'
    'A25580', # "An ansvver to" sermons
    'A67000', # "Against the doctrine of" a preacher in his sermons 
    'A41398', # Letter "modestly accepting the challenge by him made in his sermon of repentance preached"
    'A37425', # narrative account w/ "divers familiar letters, both Latin and English sermons, poems, essays"; pilgrimage
    'B29264', # dissertation upon water-baptism 
    'A60334', # response to a sermon  
    'A36211', # response to a sermon  
    'A53674', # response to a sermon  
    'A16999', # response to a sermon 
    'A71053', # response to a sermon 
    'A27407', # response to a sermon 
    'A32910', # response to a sermon; "The female advocate...Being reflections on a late rude and disingenuous discourse, delivered by Mr. John Sprint, in a sermon at a wedding"
    'A80756', # the promised "annexed sermon" is not found in the text; conversation between a minister and a converted recusant  
    'A68566', # response to a sermon; "A briefe discouery of the vntruthes and slanders...contained in a sermon"
    'A26579', # response to a sermon 
    'A76800', # contains a response to a sermon 
    'A79931', # response to a sermon; biblical commentary responding to certain writers 
    'A68078', # includes a response to a sermon
    'A04207', # responses to divines 
    'A56659', # response to a sermon
    'A53040', # response to a slanderous sermon 
    'A42574', # answer to a text 
    'A01006', # response to W. Crashawe's sermon 
    'A19895', # response to a sermon 

]

## Exclude

In [330]:
missing_df = []
sermons_df = []
excluded_df = []
excluded_annotated_df = []
foreign_df = []

for tcpID, entry in sermons.items(): 
    if tcpID in exclude_foreign: 
        foreign_df.append(entry)
    else: 
        sermons_df.append(entry)

for tcpID, entry in texts.items():
    if tcpID in ["A28224","A20529"]:
        sermons_df.append(entry) 
    elif tcpID in exclude_foreign_missing: 
        foreign_df.append(entry)
    elif tcpID in exclude_annotated: 
        excluded_annotated_df.append(entry)
    elif tcpID in exclude: 
        excluded_df.append(entry)
    else: 
        missing_df.append(entry)

missing_df = pd.DataFrame(missing_df)
missing_df.to_csv("../assets/sermons_missing.csv",index=False)
excluded_df = pd.DataFrame(excluded_df)
excluded_df.to_csv("../assets/sermons_excluded.csv", index=False)
excluded_annotated_df = pd.DataFrame(excluded_annotated_df)
excluded_annotated_df.to_csv("../assets/sermons_excluded_annotated.csv", index=False)
sermons_df = pd.DataFrame(sermons_df)
sermons_df.to_csv("../assets/sermons.csv", index=False)
foreign_df = pd.DataFrame(foreign_df)
foreign_df.to_csv("../assets/sermons_foreign.csv", index=False)

len(missing_df), len(excluded_df), len(sermons_df), len(foreign_df), len(excluded_annotated_df)

(1643, 73, 4232, 29, 52)

# Map section names to indices 

In [404]:
custom = {
        'A67822': 'poem', # poetical translation of a Latin sermon by Edward Young
        'A81042': 'letter',
        'A64811': 'class',
        'A03634': 'prefatory_letter',
        'A02189': 'address',
        'A09432': 'chapter',
        'A81871': 'section',
        'A14710': 'chapter', # (translation from German into English)
        'A35535': 'commentary_on_job',
        'A63996': 'lamentation',
        'A09434': 'verse',
        'A96523': 'decade',
        'A67068': 'religious_treatise',
        'A03628': 'funeral_oration',
        'A21269':'commentary_on_verse',
        'B03275':'colophon',
        'A60347':'funeral_sermon',
        'A89915':'commentary_on_colossians',
        'A09443':'chapter',
        'A03733':'extracts_from_sermon',
        'A02216':'oration_and_sermon',
        'A68568':'commentary_on_ruth',
        'A47013':'section',
        'A30249':'moral_treatise',
        'A36940':'section',
        'A84091':'essay', # as set forth in a sermon,
        'A34785':'catechism', # catechism adapted from a sermon
        'A17389':'biblical_commentary',
        'A02547':'panegyric',
        'A29933':'poem',
        'A64833':'class',
        'A50402':'chapter',
        'A54843':'homiletic_tract',
        'A62466':'funeral_speech',
        'A41888':'essay',
        'A14653':'chapter',
        'A61391':'chapter',
        'A46629':'speech', # Baptist speech at his trial
        'A87497':'speech', # a dissenting preacher's speech at his trial 
        'A45240':'exposition_of_job',
        'A88582':'scaffold_speech', # Christopher Love's own funeral sermon
        'B22909':'treatises',
        'A10650':'verse',
        'A46825':'funeral_sermon',
        'A43425':'theological_discourse',
        'B09423':'speech',
        'A01979':'exegesis',
        'A35389':'biblical_commentary',
        'A04154':'section',
        'A12524':'commentary_on_acts_8',
        'A33528':'polemic',
        'A01956':'biblical_commentary',
        'B27727':'chapter',
        'A04378':'section',
        'A81950':'preface', # 'chapter' is DIV2 under 'preface'
        'A18606':'book', 
        'A30167':'book', # Bunyan The Light for Those in Darkness
        'A65306':'consolatio',
        'A77994':'collection',
        'A60609':'religious_tract',
        'A12399':'religious_tract',
        'A12389':'religious_tract',
        'A12376':'religious_tract',
        'A68747':'book',
        'A80630':'book',
        'A00392':'book', # Erasmus!
        'A12369':'religious_tract',
        'A08586':'book',
        'A96415':'text', # "Manifested by letters from such as preacht to them there."   
        'A61300':'text',
        'A96951':'tract',
        'A49510': "tract", # "in a sermon preached"
        'A44593':'text', # "Delivered in a sermon"
        'A09055':'text', # 'begunne in a visitation sermon, whereunto are added the substance of divers other sermons and treatises'
        'A07647':'treatise', # "First deliuered in a sermon"
        'A28659':'treatise', # "delivered in severall sermons"
        'A19282':'treatise', # "Expressed in diuers profitable sermons"
        'A49701':'text', # collection of Hugh Latimer's sermons; subject heading 'Preaching'
        'A81043':'text', # "Being the summe and substance of certain sermons preached"
        'A16317':'text', # "deliuered in certaine sermons" 
        'A70858':'text', # "explained, confirmed, and applyed (very briefly) in one sermon"
        'A53271':['doctrine','application'], # "delivered in several sermons"
        'B18418':'discourse', # "repeated and improved in two sermons on Mat. 26. 26"
        'A16506':'text', # "Deliuered first in briefe, in a sermon preached at Paules-Crosse"
        'A07190':'text', # 'Deliuered in a sermon preached'
        'A32891':'tract', # "delivered in several sermons, and now published for publick benefit"
        'A04112':'tract', # "First deliuered in seuerall sermons"
        'B12497':'tract', # "deliuered in a sermon preached"
        'A26830':'text', # "asserted in a sermon preached"
        'A20528':'treatise', # 'preached in divers sermons, the substance whereof, is now published'
        'A61010':'text', # "delivered in a sermon at Truro in Cornwall at his primary visitation"
        'A05347':'treatise', # "The summe wherof was delivered in a sermon preached at Belfast, at the visitation of the diocese of Downe and Conner"
        'A09970':'treatise', # "The former delivered in sundry sermons in Cambridge, for the weekely fasts, 1625"
        'A21258':'text', # "deliuered in diuers sermons"
        'A09462':'text', # "in diuers sermons further manifested"
        'A58940':'text', # "in a sermon, preparatory to the Lord's-Supper"
        'A10929':'exposition', # "Deliuered in sundry sermons"; exclude the 'analysis' section 
        'A88806':'text', # "delivered in two sermons...in a third sermon at the same church"
        'A78088':'tract', # "delivered by him in a late sermon"
        'A81217':'text', # 'delivered in a funeral sermon'
        'A06732':'text', # "Delivered in sundry sermons"
        'A49757':'text', # "Preached in several sermons"
        'A01883':'part', # "First preached in a sermon"
        'A08482':'text', # 'in a sermon preached at the funerall'; ignore the 'elegy' and 'epitaph' sections by another D.D.
        'A04483':['text','treatise'], # "Both which he deliuered in diuers sermons"
        'A51907':'commentary', # "delivered in sundry sermons preacht in the church of St. James Garlick-hith London"
        'A73751':'tract', # "deliuered in certaine sermons"
        'A50410':'text', # "Certain sermons and letters"; ignore the two 'part' sections 
        'A15010':'tract', # "First preached in the parish church of Banbury in certaine sermons"
        'A13083':'text', # 'begunne in sermons, and now digested into a treatise'
        'A77775':'text', # "Preached in Concord in Nevv-England"
        'A96416':'text', # 'preached there, and elsewhere'
        'A48424':'tract', # 'Preached by"
        'A54347':'discourse', # 'first preached, and now published" 
        'A37030':'text', # "Preached at Reading" 
        'A03342':'part', # "CVIII lectures vpon the fourth of Iohn Preached at Ashby-Delazouch"
        'A06160':'part', # "First preached, and now published" 
        'A06161':'part', # "First preached, and now published"
        'B43800':'text', #  "First preached and afterwards printed"
        'A19614':'exposition', # 'First preached in his Parish Church'
        'A16166':'text', # "First preached, then penned"
        'A13209':'text', # "Preached by"
        'A66772':'part', # "warrantably proclaimed and preached by Geo. Wither"; subject heading "History"
        'A09465':'treatise', # "Framed and preached by M. Wil. Perkins."
        'A50026':'text', # "Since occasionally preached in part...and now presented to publick view,"
        'A29121':'text', # "By Thomas Bradley doctor of divinity, chaplaine to His late Majesty King Charles the First, and præbend of York. And there preached at Lent assizes holden there"
        'A62878':'treatise', # 'preached at Lemster'
        'A38609':['part','commandment'], # New observations upon the decalogue: or The second of the four parts of Christian doctrine, preached upon the catechism. 
        'A44674':'text', # "Preached to a country congregation, by J.H. And publish'd by one who wrote it from his mouth."
        'A10557':'text', # BOOK OF COMMON PRAYER! Edmund Reeve
        'A38608':'text', # "New observations upon the Creed, or, The first of the four parts of the doctrine of Christianity preached upon the catechism of the French churches"
        'A04626':'text', # "A treatise of patience in tribulation first, preached before..."; ignore the "part" and "epicedium" 
        'A09463':'text', # "A commentarie vpon the temptations of Christ: preached in Cambridge by that reuerend and iudicious diuine M. William Perkins"
        'A25215':'text', # "partly preached at Guild-hall Chappel"
        'A76324':'text', # A man's last words; "With a sermon made upon this text, preached at his funeral by a reverend divine, Mr. Thomas Palmer."

        'A95937':'vindication', # <DIV1 TYPE="vindication"><PB N="21" REF="13"/> # the 3rd vindication
        'A13078':'book', # only the first one; the second one is a discourse expanding on his sermon
        'A80317':'text', # <DIV1 TYPE="text"><PB REF="2"/> Contains the substance of the sermon preached to them
        'A69511':'part', # the SECOND part -- exclude the contents within the <LETTER></LETTER> and <LIST></LIST>. Begin with the following: <P><PB N="13" REF="8"/>
}

In [370]:
# r"^sermon"
wanted_sections = [
    'funeral_sermon',
    'extracts_from_sermon',
    'oration_and_sermon',
    'collection_of_lectures',
    'collection_of_sermons_on_isaiah',
    'collection_of_sermons_on_haggai',
    'whit_sunday_sermons',
    'ordination_sermons',
    'penitential_sermons_preached_at_wells',
    'sermon_extract',
    'application_of_sermon',
    'summary_of_sermons',
    'two_sermons',
    'greek_text_bound_with_sermon',
    'collection_of_sermons',
    'visitation_sermon',
    'lecture',
    'lectures'
]

In [407]:
import os,re
import pandas as pd 
from collections import Counter 

sermons_missing = pd.read_csv("../assets/sermons_missing.csv")
sermons_missing = sermons_missing.to_dict(orient='records')
sermons_missing = {s['id']: s for s in sermons_missing}

num_oral = 0 
sections = [] # tcpID to section to index 

for fp in os.listdir("../assets/plain_all"): 
    if fp == ".DS_Store": continue 
    tcpID = fp.split(".")[0]
    
    if tcpID not in sermons and tcpID not in sermons_missing: 
        continue 
    if tcpID in sermons: continue 
    
    with open(f"../assets/plain_all/{fp}","r") as file: 
        text = file.read() 
    s = re.findall(r"SECTION(\d+):(\w+)\b",text)
    
    skip = False 
    tcpID_sections = []
    title = sermons_missing[tcpID]['title']
    for _, name in s: 
        if re.search(r"^sermon",name): 
            skip = True
        elif tcpID in custom: 
            skip = True 
        elif tcpID in exclude or tcpID in exclude_foreign_missing or tcpID in exclude_annotated: 
            skip = True 
        elif re.search('Sermons|Funeral sermons|Thanksgiving sermons|Fast-day sermons|Christmas sermons|Visitation sermons',sermons_missing[tcpID]['subject_headings']):
            skip = True 
        elif re.search("lecture",name) and re.search(r"preach(?:'d|t|ed)",title): 
            skip = True
        elif re.search(r"sermons preach(?:'d|t|ed)",title) and re.search(r"substance|summe",title): 
            skip = True 
        # elif not re.search(r"in\s+(\w+\s+){0,2}sermon",title): 
        #     skip = True 
        elif re.search(r"lecture",title.lower()) and re.search(r"preach(?:'d|t|ed)",title.lower()): 
            skip = True  
        elif re.search(r"preach(?:'d|t|ed)",title.lower()) and re.search(r"publish(?:'d|t|ed)|penned",title.lower()): 
            skip = True  
        elif not re.search(r"concerning|reflections|answer|ansvver",title):
            skip = True  
        # elif name in ['text','lecture','treatise','tract','homily',
        #               'part','body_of_text','discourse',
        #               'exposition','commentary','memorial',
        #               'doctrine']: 

        elif not skip: 
            tcpID_sections.append(name)
    if skip: continue 
    print(tcpID, Counter(tcpID_sections), "-----------",
          sermons_missing[tcpID]['authors'],"-----------",
          sermons_missing[tcpID]['title'],"-----------",
          sermons_missing[tcpID]['subject_headings'])
# _________________
        # for idx, name in s: 
        #     if re.search('speech|oratio|homil|eulog|lecture',name): 
        #         print(tcpID, name)
        #         print(sermons_missing[tcpID]['title'])
        #         print(sermons_missing[tcpID]['subject_headings'])
        #         print()
        #         break
    # s = re.findall(r"SECTION(\d+):(\w+)\b",text)
    # for idx, name in s: 
    #     if re.search(r"sermon|speech|oratio|homil|eulog|lecture|encomi|exhortation|memorial|consolatio",name):
    #         num_oral += 1  
    #     sections.append({"tcpID":tcpID, "section_idx":idx,"section_name":name})
# sections = pd.DataFrame(sections)
# sections.to_csv('/Users/amycweng/DH/SERMONS_APP/db/data/sections.csv',header=False,index=False)
# print(num_oral)
# sections

A04214 Counter({'part': 2, 'title_page': 1, 'dedication': 1, 'preface': 1, 'errata': 1, 'chart': 1}) ----------- Jacob, Henry, 1563-1624. ----------- A defence of a treatise touching the sufferings and victorie of Christ in the worke of our redemption Wherein in confirmed, 1 That Christ suffered for vs, not only bodily griefe, but also in his soule an impression of the proper wrath of God, which may be called the paines of Hell. 2 That after his death on the crosse he went not downe into Hell. For answere to the late writings of Mr Bilson, L. Bishop of Winchester, which he intitleth, The effect of certaine sermons, &c. Wherein he striueth mightly against the doctrine aforesaid. By Henry Iacob minister of the worde of God. ----------- Redemption -- Bilson, Thomas, 1546 or 7-1616 --  Effect of certain sermons, &c. --  Early works to 1800.
A27593 Counter({'title_page': 1, 'text': 1, 'publisher': 1}) ----------- Beverley, Thomas. ----------- A conciliatoy [sic] judgment concerning Dr. Cris

In [371]:
for name, _ in Counter(sections).items(): 
    if not re.search(r"^sermon",name): 
        if name not in wanted_sections: 
            print(name, _)


list_of_published_sermons 1
list_of_sermons 1
catalogue_of_sermons_preached_before_parliament 1
advert_for_lecture_series 1


In [344]:
for tcpID in ['A20529']: 
    print(sermons_missing[tcpID]['authors'])
    print(sermons_missing[tcpID]['title'])
    print(sermons_missing[tcpID]['subject_headings'])
    print(sermons_missing[tcpID]['date'])
    with open(f"../assets/plain_all/{tcpID}.txt","r") as file: 
        text = file.read() 
        s = re.findall(r"SECTION(\d+):(\w+)\b",text)
        print(s)
        print(text[:2000])

Greenham, Richard.; Cleaver, Robert, 1561 or 2-ca. 1625.; Winston, John, fl. 1614-1634.; Dod, John, 1549?-1645.
Foure godlie and fruitful sermons two preached at Draiton in Oxford-shire, at a fast, enioyned by authoritie, by occasion of the pestilence then dangerously dispearsed. Likewise two other sermons on the twelfth Psalme. VVhereunto is annexed a briefe tract of zeale. / By I. Dod. R. Cleauer.
 Early works to 1800. --  17th century. -- Sermons, English --  Sermons -- Plague --  England
1611
[('0', 'title_page'), ('1', 'dedication'), ('2', 'points_of_doctrine'), ('3', 'sermon'), ('4', 'sermon'), ('5', 'sermon'), ('6', 'sermon'), ('7', 'tract')]
 SECTION0:title_page PAGEIMAGE1 PAGEIMAGE1 STARTPARAGRAPH0 FOVRE GODLIE AND FRVITFVL SERMONS: TWO PREACHED AT DRAITON IN OXFORD-SHIRE, At a Fast, enioyned by authoritie, by occasion of the pestilence then dangerously dispearsed. STARTPARAGRAPH1 Likewise STARTPARAGRAPH2 TWO OTHER SERMONS ON the twelfth Psalme. STARTPARAGRAPH3 STARTITALICS VV

In [193]:
for tcpID in ['A28224']: 
    with open(f"../assets/plain_all/{tcpID}.txt","r") as file: 
        text = file.read() 
        s = re.findall(r"SECTION(\d+):(\w+)\b",text)
        print(s)
        print(text[:1000])

[('0', 'title_page'), ('1', 'dedication'), ('2', 'title_page'), ('3', 'sermon'), ('4', 'title_page'), ('5', 'sermon'), ('6', 'publisher')]
 SECTION0:title_page PAGEIMAGE1 STARTPARAGRAPH0 TWO SERMONS. THE FIRST, Shewing the STARTITALICS Mischiefs ENDITALICS of ANARCHY. THE SECOND, STARTITALICS The Mischiefs of SEDITION. ENDITALICS AND BOTH OF THEM, STARTITALICS The Mischiefs and Treasons ENDITALICS OF CONVENTICLES. STARTPARAGRAPH1 PREACHED At the ASSIZES held for the County of STARTITALICS SFFOLK, Ann. ENDITALICS 1682/3. STARTITALICS AND PBLISHED ENDITALICS At the request of STARTITALICS THO. WALDEGRAVE, ENDITALICS Esq High Sheriff of the said County. STARTPARAGRAPH2 By STARTITALICS NATH. BISBIE, ENDITALICS D. D. STARTPARAGRAPH3 STARTITALICS LONDON, ENDITALICS Printed for STARTITALICS Walter Kettilby, ENDITALICS at the STARTITALICS Bishop ENDITALICS 's STARTITALICS Head ENDITALICS in St. STARTITALICS Paul ENDITALICS 's Church-Yard, MDCLXXXIV.   SECTION1:dedication PAGEIMAGE2 PAGEIMAGE2 

In [6]:
for fp in ['B17774','A52900','B03275','A22477']: 
    with open(f"../assets/plain_all/{fp}.txt","r") as file: 
        text = file.read()
        print(text[:200])

 SECTION0:title_page PAGEIMAGE1 STARTPARAGRAPH0 THE WHOLE BOOK OF PSALMS: Collected into English Metre, BY STARTITALICS Thomas Sternhold, John Hopkins, ENDITALICS and others. STARTPARAGRAPH1 Set forth
 SECTION0:answer PAGE1 SOME REMARKS On Mr. Bois Book In Defence of Osborn. And upon some PASSAGES in Mr. Williams SERMON on the 23d. of October last. Sent in a Letter to satisfie his Friend, a Dissent
 SECTION0:elegy PAGEIMAGE1 AN ELEGY Upon the DEATH of TWO Eminent Ministers of the Gospel Mr. PLEDGER and Mr. WELLS. Who Both Departed this Life on the Lords Day June the 18th. 1676. The First of them
 SECTION0:royal_proclamation PAGEIMAGE1 C R DIEV ET MON DROIT HONI SOIT QVI MAL Y PENSE royal blazon or coat of arms By the King. A Proclamation for the calling in, and suppressing of two Sermons, Pre
